In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.simplefilter(action='ignore')

## Load dataset

In [2]:
pd.pandas.set_option('display.max_columns', None)
data = pd.read_csv('../data/raw/houseprices.csv')
data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,Gas

## Split dataset into train and test set

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, data['SalePrice'], test_size=0.1, random_state=0)

print("Train set:", X_train.shape)
print("Test set:", X_test.shape)

Train set: (1314, 81)
Test set: (146, 81)


## Missing values in numerical variables

In [4]:
# Create a list with numerical variables that contain missing value

num_variables_with_nan = []

for variable in data.columns:
    if X_train[variable].isnull().sum() > 0 and X_train[variable].dtypes != 'O':
        num_variables_with_nan.append(variable)

# Share of null values
X_train[num_variables_with_nan].isnull().mean()

LotFrontage    0.177321
MasVnrArea     0.004566
GarageYrBlt    0.056317
dtype: float64

In [5]:
# Replace missing values by the mode

for variable in num_variables_with_nan:
    mode_value = X_train[variable].mode()[0]
    X_train[variable] = X_train[variable].fillna(mode_value)
    X_test[variable] = X_test[variable].fillna(mode_value)

# Share of null values after replacement
X_train[num_variables_with_nan].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

## Missing values in categorical variables

In [6]:
# Create a list with categorical variables that contain missing value

cat_variables_with_nan = []

for variable in data.columns:
    if X_train[variable].isnull().sum() > 0 and X_train[variable].dtypes == 'O':
        cat_variables_with_nan.append(variable)

# Share of null values
X_train[cat_variables_with_nan].isnull().mean()

Alley           0.938356
MasVnrType      0.004566
BsmtQual        0.024353
BsmtCond        0.024353
BsmtExposure    0.025114
BsmtFinType1    0.024353
BsmtFinType2    0.025114
Electrical      0.000761
FireplaceQu     0.472603
GarageType      0.056317
GarageFinish    0.056317
GarageQual      0.056317
GarageCond      0.056317
PoolQC          0.995434
Fence           0.814307
MiscFeature     0.961187
dtype: float64

In [7]:
# Replace missing values with new label: "Missing"

X_train[cat_variables_with_nan] = X_train[cat_variables_with_nan].fillna('Missing')
X_test[cat_variables_with_nan] = X_test[cat_variables_with_nan].fillna('Missing')

# Share of null values after replacement
X_train[cat_variables_with_nan].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

## Temporal variables

In [8]:
# Get the time elapsed between variable and the year in which the house was sold

temporal_variables = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

for variable in temporal_variables:
    X_train[variable] = X_train['YrSold'] - X_train[variable]
    X_test[variable] = X_test['YrSold'] - X_test[variable]

In [9]:
X_train[temporal_variables].sort_index().head(5)

,YearBuilt,YearRemodAdd,GarageYrBlt
0,5,5,5.0
1,31,31,31.0
2,7,6,7.0
3,91,36,8.0
6,3,2,3.0


## Numerical variable transformation

In [10]:
# Log transformation of non-normal distributed variables

non_normal_dist_variables = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']

for variable in non_normal_dist_variables:
    X_train[variable] = np.log(X_train[variable])
    X_test[variable] = np.log(X_test[variable])

In [11]:
X_train[non_normal_dist_variables].sort_index().head(5)

,LotFrontage,LotArea,1stFlrSF,GrLivArea,SalePrice
0,4.174387,9.041922,6.752270,7.444249,12.247694
1,4.382027,9.169518,7.140453,7.140453,12.109011
2,4.219508,9.328123,6.824374,7.487734,12.317167
3,4.094345,9.164296,6.867974,7.448334,11.849398
6,4.317488,9.218705,7.434848,7.434848,12.634603


## Rare labels in categorical variables

In [12]:
# Replace rare labels (which appear only in a small proportion of the observations) by the string "Rare"

categorical_variables = []

for variable in X_train.columns:
    if data[variable].dtypes == 'O':
        categorical_variables.append(variable)    

In [13]:
def find_non_rare_labels(df, variable, tolerance):
    df = df.copy()
    temp = df.groupby(variable)['SalePrice'].count() / len(df)
    return temp[temp > tolerance].index

In [14]:
for variable in categorical_variables:
    frequent_category = find_non_rare_labels(X_train, variable, 0.01)
    
    X_train[variable] = np.where(X_train[variable].isin(frequent_category), X_train[variable], 'Rare')
    X_test[variable] = np.where(X_test[variable].isin(frequent_category), X_test[variable], 'Rare')

## Categorical variable encoding

In [15]:
# Transform the strings of the categorical variables into numbers

def replace_categories(train, test, variable, target):
    
    # organize categories according to target mean
    ordered_labels = train.groupby([variable])[target].mean().sort_values().index
    
    # create the dictionary to convert ordered labels to ordinal number
    dict_ordered_labels = {k: i for i, k in enumerate(ordered_labels, 0)}
    
    # convert categories to these ordinal numbers
    train[variable] = train[variable].map(dict_ordered_labels)
    test[variable] = test[variable].map(dict_ordered_labels)

In [16]:
for variable in categorical_variables:
    replace_categories(X_train, X_test, variable, 'SalePrice')

In [17]:
X_train[categorical_variables].sort_index().head(10)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3,1,2,0,1,1,0,0,15,2,1,3,5,0,0,10,10,2,2,3,4,3,3,1,6,4,2,4,1,3,2,4,1,4,2,2,3,2,0,3,2,2,3
1,3,1,2,0,1,1,1,0,11,1,1,3,3,0,0,3,2,0,1,3,2,3,3,4,4,4,2,4,1,3,1,4,3,4,2,2,3,2,0,3,2,2,3
2,3,1,2,1,1,1,0,0,15,2,1,3,5,0,0,10,10,2,2,3,4,3,3,2,6,4,2,4,1,3,2,4,3,4,2,2,3,2,0,3,2,2,3
3,3,1,2,1,1,1,2,0,16,2,1,3,5,0,0,1,3,0,1,3,1,2,4,1,4,4,2,3,1,3,2,4,4,2,1,2,3,2,0,3,2,2,1
6,3,1,2,0,1,1,0,0,18,2,1,3,3,0,0,10,10,3,2,3,4,4,3,3,6,4,2,4,1,3,2,4,4,4,2,2,3,2,0,3,2,2,3
7,3,1,2,1,1,1,2,0,12,5,1,3,5,0,0,6,6,3,1,3,2,3,3,2,4,1,2,4,1,3,1,4,3,4,2,2,3,2,0,3,1,2,3
8,1,1,2,0,1,1,0,0,5,0,1,3,1,0,0,8,3,0,1,3,1,2,3,1,5,4,2,3,1,1,1,3,3,2,1,1,3,2,0,3,2,2,1
10,3,1,2,0,1,1,0,0,6,2,1,3,3,2,0,6,6,0,1,3,2,2,3,1,1,4,2,4,1,3,1,4,1,2,1,2,3,2,0,3,2,2,3
11,3,1,2,1,1,1,0,0,21,2,1,3,5,2,0,2,3,3,3,3,4,4,3,1,6,4,2,4,1,3,3,4,4,5,3,2,3,2,0,3,2,3,4
12,3,1,2,3,1,1,0,0,6,2,1,3,3,2,0,6,7,0,1,3,2,2,3,1,4,4,2,2,1,3,1,4,1,2,1,2,3,2,0,3,2,2,3


## Feature scaling

In [18]:
# Scaling variables to the minimum and maximum values (MinMaxScaler)

variables_to_scale = [variable for variable in X_train.columns if variable not in ['Id', 'SalePrice']]

scaler = MinMaxScaler()

scaler.fit(X_train[variables_to_scale])

X_train[variables_to_scale] = scaler.transform(X_train[variables_to_scale])
X_test[variables_to_scale] = scaler.transform(X_test[variables_to_scale])

In [19]:
X_train.sort_index().head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,0.235294,0.75,0.418208,0.366344,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.681818,0.4,1.0,0.75,1.0,0.666667,0.500,0.036765,0.098361,0.0,0.0,1.0,1.0,0.666667,0.12250,0.666667,1.0,1.00,0.75,0.75,0.25,1.000000,0.125089,0.666667,0.00000,0.069670,0.140098,1.0,1.00,1.0,1.000000,0.356155,0.456197,0.0,0.577712,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.00,0.000000,0.2,0.8,0.046729,0.666667,0.50,0.386460,0.666667,1.0,1.0,0.000000,0.111517,0.000000,0.0,0.000000,0.0,0.0,0.75,1.0,0.000000,0.090909,0.50,0.666667,0.75,12.247694
1,2,0.000000,0.75,0.495064,0.391317,1.0,1.0,0.000000,0.333333,1.0,0.25,0.0,0.500000,0.2,1.0,0.75,0.6,0.555556,0.875,0.227941,0.524590,0.0,0.0,0.3,0.2,0.000000,0.00000,0.333333,1.0,0.50,0.75,0.75,1.00,0.666667,0.173281,0.666667,0.00000,0.131909,0.206547,1.0,1.00,1.0,1.000000,0.503056,0.000000,0.0,0.470245,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,1.00,0.333333,0.6,0.8,0.289720,0.666667,0.50,0.324401,0.666667,1.0,1.0,0.347725,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.75,1.0,0.000000,0.363636,0.25,0.666667,0.75,12.109011
2,3,0.235294,0.75,0.434909,0.422359,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.681818,0.4,1.0,0.75,1.0,0.666667,0.500,0.051471,0.114754,0.0,0.0,1.0,1.0,0.666667,0.10125,0.666667,1.0,1.00,0.75,0.75,0.50,1.000000,0.086109,0.666667,0.00000,0.201579,0.150573,1.0,1.00,1.0,1.000000,0.383441,0.462607,0.0,0.593095,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.00,0.333333,0.6,0.8,0.065421,0.666667,0.50,0.428773,0.666667,1.0,1.0,0.000000,0.076782,0.000000,0.0,0.000000,0.0,0.0,0.75,1.0,0.000000,0.727273,0.50,0.666667,0.75,12.317167
3,4,0.294118,0.75,0.388581,0.390295,1.0,1.0,0.333333,0.333333,1.0,0.50,0.0,0.727273,0.4,1.0,0.75,1.0,0.666667,0.500,0.669118,0.606557,0.0,0.0,0.1,0.3,0.000000,0.00000,0.333333,1.0,0.25,0.50,1.00,0.25,0.666667,0.038271,0.666667,0.00000,0.250813,0.123732,1.0,0.75,1.0,1.000000,0.399941,0.403846,0.0,0.579157,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.00,0.333333,0.8,0.4,0.074766,0.333333,0.75,0.452750,0.666667,1.0,1.0,0.000000,0.063985,0.492754,0.0,0.000000,0.0,0.0,0.75,1.0,0.000000,0.090909,0.00,0.666667,0.25,11.849398
6,7,0.000000,0.75,0.471175,0.400943,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.818182,0.4,1.0,0.75,0.6,0.777778,0.500,0.022059,0.049180,0.0,0.0,1.0,1.0,1.000000,0.11625,0.666667,1.0,1.00,1.00,0.75,0.75,1.000000,0.242558,0.666667,0.00000,0.147236,0.275941,1.0,1.00,1.0,1.000000,0.614465,0.000000,0.0,0.574387,0.333333,0.0,0.666667,0.0,0.375,0.333333,0.666667,0.416667,1.00,0.333333,0.8,0.8,0.028037,0.666667,0.50,0.448519,0.666667,1.0,1.0,0.297550,0.104205,0.000000,0.0,0.000000,0.0,0.0,0.75,1.0,0.000000,0.636364,0.25,0.666667,0.75,12.634603
7,8,0.235294,0.75,0.388581,0.406643,1.0,1.0,0.333333,0.333333,1.0,0.50,0.0,0.545455,1.0,1.0,0.75,1.0,0.666667,0.625,0.264706,0.606557,0.0,0.0,0.6,0.6,1.000000,0.15000,0.333333,1.0,0.50,0.75,0.75,0.50,0.666667,0.152197,0.166667,0.02171,0.100325,0.181178,1.0,1.00,1.0,1.000000,0.453465,0.525107,0.0,0.648700,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.333333,0.416667,1.00,0.666667,0.6,0.8,0.336449,0.666667,0.50,0.341326,0.666667,1.0,1.0,0.274212

## Saving processed train and test sets

In [20]:
#X_train.to_csv('../data/processed/train_processed.csv', index=False)
#X_test.to_csv('../data/processed/test_processed.csv', index=False)